## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [2]:
# Подключим нужные нам библиотеки для работы с данными и выведем методом df.info() информацию о датафрейме
import pandas as pd
from pymystem3 import Mystem
m = Mystem()

df = pd.read_csv('data.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [3]:
#Так как данный столбец нам будет необходим при дальнейшей работе, Посчитаем категории в столбце children, 
df['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

In [4]:
#Посчитаем категории в столбце education
df['education'].value_counts().sort_index()

ВЫСШЕЕ                   274
Высшее                   268
НАЧАЛЬНОЕ                 17
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
Начальное                 15
Неоконченное высшее       47
СРЕДНЕЕ                  772
Среднее                  711
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
высшее                  4718
начальное                250
неоконченное высшее      668
среднее                13750
ученая степень             4
Name: education, dtype: int64

In [5]:
#Также посчитаем категории в столбце Family_status
df['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

In [6]:
df['purpose'].value_counts()

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
операции с жильем                         653
покупка жилья для сдачи                   653
операции с коммерческой недвижимостью     651
жилье                                     647
покупка жилья                             647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
покупка своего жилья                      620
строительство недвижимости                620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

### Вывод

1) Из информации методом info() мы видим, что в датафрейме имеются пустые значения. 

2) В столбце children имеются отрицательные значения, и немного смущает что у некоторых семей по 20 детей. Возможно эти ошибки допустили при вводе данных.

3) Исходя из данных столбца education, мы видим, что больше всего получали кредит люди со средним образованием. Также образование имеет проблемы с регистром, что создает проблемы для категоризации. необходима перевести все в нижний регистр.

4) Из столбца family_status мы видим, что больше всего кредиты получили женатые/замужние. Так же нужно привести к нижнему регистру.

5) В столбце purpose мы видим, похожие цели по смыслу на получение кредита. Необходимо исправить с помощью лемматизации.


### Шаг 2. Предобработка данных

### Обработка пропусков

In [7]:
#Проверим датафрейм на пустые значения
df.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [8]:
#заменим пустые значение на 0
df=df.fillna(0)

In [9]:
#перепроверим датафрейм на пустые значения
df.isnull().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

In [10]:
#В столбце children необходимо убрать отрицательные значения
df = df[(df['children'] >= 0) & (df['children'] <= 20)]
df['children'].value_counts() #проверим, убрали ли мы отрицательные значения

0     14149
1      4818
2      2055
3       330
20       76
4        41
5         9
Name: children, dtype: int64

In [11]:
#проверка на пустые значения
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21478 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21478 non-null  int64  
 1   days_employed     21478 non-null  float64
 2   dob_years         21478 non-null  int64  
 3   education         21478 non-null  object 
 4   education_id      21478 non-null  int64  
 5   family_status     21478 non-null  object 
 6   family_status_id  21478 non-null  int64  
 7   gender            21478 non-null  object 
 8   income_type       21478 non-null  object 
 9   debt              21478 non-null  int64  
 10  total_income      21478 non-null  float64
 11  purpose           21478 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.1+ MB


### Вывод

Пустые значения имелись в столбцах days_employed и total_income, поэтому для более удобного расчета мы заполнили нулем все пустые значения методом fillna().
Скорее всего эти пустые значения были внесены преднамеренно. Безработный не имеет дохода. соответственно два столбца были пустыми. 

### Замена типа данных

In [12]:
# Начнем с работы над регистрами, приведем все нижнему к регистру.
df['education']=df['education'].str.lower()
df['family_status']=df['family_status'].str.lower()
df['family_status'].value_counts()

женат / замужем          12351
гражданский брак          4172
не женат / не замужем     2808
в разводе                 1191
вдовец / вдова             956
Name: family_status, dtype: int64

In [13]:
df['education'].value_counts()

среднее                15196
высшее                  5251
неоконченное высшее      743
начальное                282
ученая степень             6
Name: education, dtype: int64

In [14]:
# изменим тип данных с float на int методом .astype(int)
df['days_employed'] = df['days_employed'].astype(int)
df['total_income'] = df['total_income'].astype(int)

In [15]:
#Проверим, удалось ли нам изменить тип данных
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21478 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21478 non-null  int64 
 1   days_employed     21478 non-null  int64 
 2   dob_years         21478 non-null  int64 
 3   education         21478 non-null  object
 4   education_id      21478 non-null  int64 
 5   family_status     21478 non-null  object
 6   family_status_id  21478 non-null  int64 
 7   gender            21478 non-null  object
 8   income_type       21478 non-null  object
 9   debt              21478 non-null  int64 
 10  total_income      21478 non-null  int64 
 11  purpose           21478 non-null  object
dtypes: int64(7), object(5)
memory usage: 2.1+ MB


### Вывод

1) привели столбец education, в котором имели проблемы с регистрами к нижнему регистру.

2) Изменили тип данных с float на int у столбцов days_employed и total_income, методом astype(int)



### Обработка дубликатов

In [16]:
#проверка на наличиедубликатов
df.duplicated().sum()

71

In [17]:
#Удалим эти дубликаты.
df = df.drop_duplicates().reset_index(drop=True)

In [18]:
#проверим еще раз на наличие дубликатов
df.duplicated().sum()

0

In [19]:
#Перейдем к ручному поиску дубликатов в столбце purpose
df['purpose'].value_counts().sort_index()

автомобили                                477
автомобиль                                492
высшее образование                        451
дополнительное образование                458
жилье                                     645
заняться высшим образованием              496
заняться образованием                     408
на покупку автомобиля                     470
на покупку подержанного автомобиля        477
на покупку своего автомобиля              504
на проведение свадьбы                     767
недвижимость                              632
образование                               444
операции с жильем                         651
операции с коммерческой недвижимостью     649
операции с недвижимостью                  673
операции со своей недвижимостью           624
покупка жилой недвижимости                604
покупка жилья                             643
покупка жилья для сдачи                   650
покупка жилья для семьи                   638
покупка коммерческой недвижимости 

### Вывод

1) Нашли дубликаты в нашем датафрейме методом duplicates()

2) Удалили дубликаты с помощью drop.duplicates() изменив при этом индекс

3) С помощью ручного поиска дубликатов нашли одинаковые по смыслу цели, которые также нужно будет привести к одной категории. Например покупка жилья и покупка жилья для семьи можно привести к одной категории, чтобы было легче с ними работать.


### Лемматизация

In [20]:
#напишем функцию для лемматизации столбца purpose 

def purpose_lemma(row):
    lemmas = m.lemmatize(row)
    for word in lemmas:
        try:
            if 'свадьба' in word:
                return 'свадьба'
        
            if 'недвижимость' in word:
                return 'недвижимость'
        
            if 'жилье' in word:
                return 'недвижимость'
        
            if 'автомобиль' in word:
                return 'автомобиль'
        
            if 'образование' in word:
                return 'образование'

        except:
            print('Error in lemmatization')


In [21]:
#добавим в новый столбец уже разбросанные по нашим категориям данные
df['grouped_purpose'] = df['purpose'].apply(purpose_lemma)

In [22]:
#проверим какие категории теперь у нас имеются
df['grouped_purpose'].value_counts()

недвижимость    10787
автомобиль       4295
образование      4003
свадьба          2322
Name: grouped_purpose, dtype: int64

### Вывод

С помощью лемматизации мы определили все цели по 4 категориям : свадьба, недвижимость, автомобиль и образование. 

### Категоризация данных

In [23]:
#  в переменную total_income_median назначим медиану столбца total_income
total_income_median = df['total_income'].median() 
total_income_median

135760.0

In [24]:
#Напишем функцию для распределения total_income по категориям с использованием юмеющейся медианы. 
#Данные заработной платы взяли по России за 2019г. средняя зп 37900 рублей и средняя высокая зп 100 000 рублей.

def group_total_income(total_income):
    if total_income < 37900:
        return 'низкий доход'
    if 37900 < total_income < total_income_median:
        return 'средний доход'
    if  total_income_median < total_income < 200000:
        return 'доход выше среднего'
    else:
        return 'высокий доход'

In [25]:
df['grouped_total_income'] = df['total_income'].apply(group_total_income)

In [26]:
df['grouped_total_income'].value_counts()

средний доход          8508
доход выше среднего    5645
высокий доход          5059
низкий доход           2195
Name: grouped_total_income, dtype: int64

In [27]:
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,grouped_purpose,grouped_total_income
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость,высокий доход
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,средний доход
2,0,-5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость,доход выше среднего
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование,высокий доход
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,доход выше среднего
5,0,-926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,недвижимость,высокий доход
6,0,-2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,недвижимость,высокий доход
7,0,-152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,образование,доход выше среднего
8,2,-6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,свадьба,средний доход
9,0,-2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,недвижимость,доход выше среднего


### Вывод

1) Взяли медиану по столбцу total_income, для того чтобы использовать ее в категоризации по зарплатам. 

2) Мы взяли данные по заработной плате в России за 2019г  для использования их в  категоризации столбца. Написали функцию group_total_income для категоризации по столбцу total_income.

3) применили функцию к столбцу с помощью apply 

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [28]:
#Путем pivot_table проверим зависимость между столбцами children и debt
pd.pivot_table(df, index='children', values='debt')

,debt
children,
0,0.075438
1,0.092346
2,0.094542
3,0.081818
4,0.097561
5,0.000000
20,0.105263


### Вывод

Исходя из данных, мы видим, что кредиторы не имеющие детей, закрывают кредит чаще, чем те, у кого есть один или более детей.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [30]:
pd.pivot_table(df,index='family_status', values='debt').sort_values(by='debt',ascending=True)

,debt
family_status,
вдовец / вдова,0.065969
в разводе,0.071369
женат / замужем,0.075548
гражданский брак,0.093584
не женат / не замужем,0.097683


### Вывод

Исходя из полученных данных, мы видим что, семейное положение положительно влияет на погашение кредита в срок. Вдовец/Вдова чаще всех погашают кредит своевременно, после них люди в разводе. Чаще всех имеют задолжности с погашением кредита не женатые и не замужние.



- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [31]:
pd.pivot_table(df,index='grouped_total_income', values='debt').sort_values(by='debt',ascending=True)

,debt
grouped_total_income,
высокий доход,0.070765
низкий доход,0.079727
средний доход,0.082393
доход выше среднего,0.089637


### Вывод

У людей с высоким доходом, кредитная история лучшая, чем у людей с низким или средним доходом.

- Как разные цели кредита влияют на его возврат в срок?

In [32]:
pd.pivot_table(df, index='grouped_purpose', values='debt').sort_values(by='debt',ascending=True)

,debt
grouped_purpose,
недвижимость,0.072495
свадьба,0.080103
образование,0.092431
автомобиль,0.093597


### Вывод

Полученные данные говорят о том, что люди взявшие кредит на недвижимость, чаще своевременно погашают кредит, нежели взяв кредит на другие цели.

### Шаг 4. Общий вывод

Проведя анализ полученных мною входных данных от банка, можно прийти к выводу, что семейное положение влияет на факт погашения кредита в срок, так же, как и наличие детей. 

Люди состоявшие в браке, или состоящие на данный момент, имеют меньше задолжностей перед банком, нежели холостяки/холостячки, 
Видимо семейный брак делает людей более ответственными. 

Что касается наличия детей, то разница совсем небольшая. Всего лишь 7.5% бездетных кредиторов имеют задолжности перед банком, против 8% кредиторов с детьми. Разница есть, но не существенная. 

### Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение, какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено, по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.